In [1]:
#!pip install azure-storage-blob

#mino configuration
curl https://dl.min.io/client/mc/release/linux-amd64/mc \--create-dirs \-o $HOME/minio-binaries/mc
chmod +x $HOME/minio-binaries/mc
export PATH=$PATH:$HOME/minio-binaries/
mc config host ls
mc config host add kubeflow http://minio-service.kubeflow.svc.cluster.local:9000
kubectl -n kubeflow port-forward  --address="0.0.0.0" svc/minio-service 9000:9000
mc ls kubeflow
mc mb kubeflow/kserve-test

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor

In [3]:
# Create a random dataset
rng = np.random.RandomState(1)
X = np.sort(200 * rng.rand(100, 1) - 100, axis=0)
y = np.array([np.pi * np.sin(X).ravel(), np.pi * np.cos(X).ravel()]).T
y[::5, :] += 0.5 - rng.rand(20, 2)

In [4]:
regr_1 = DecisionTreeRegressor(max_depth=2)
regr_2 = DecisionTreeRegressor(max_depth=5)
regr_3 = DecisionTreeRegressor(max_depth=8)
regr_1.fit(X, y)
regr_2.fit(X, y)
regr_3.fit(X, y)

DecisionTreeRegressor(max_depth=8)

In [5]:
# Predict
X_test = np.arange(-100.0, 100.0, 0.01)[:, np.newaxis]
y_1 = regr_1.predict(X_test)
y_2 = regr_2.predict(X_test)
y_3 = regr_3.predict(X_test)

y_3

array([[ 1.82569761,  2.64490159],
       [ 1.82569761,  2.64490159],
       [ 1.82569761,  2.64490159],
       ...,
       [-1.84319053, -0.55258289],
       [-1.84319053, -0.55258289],
       [-1.84319053, -0.55258289]])

In [6]:
import joblib

import os
os.environ.update({
    'S3_ENDPOINT'          : 'minio-service.kubeflow:9000',
    'AWS_ACCESS_KEY_ID'    : 'minioadmin',
    'AWS_SECRET_ACCESS_KEY': 'minioadmin',
    'S3_USE_HTTPS'         : '0',   # Whether or not to use HTTPS. Disable with 0.                        
    'S3_VERIFY_SSL'        : '0'    # If HTTPS is used, controls if SSL should be enabled. Disable with 0.
})  

In [7]:
bucket_name="kserve-test"
model_path="/home/user1/model.joblib"
joblib.dump(regr_3, model_path)

['/home/user1/model.joblib']

In [8]:
!mc cp /home/user1/model.joblib kubeflow/kserve-test/sklearn-test/v1/model.joblib

/bin/bash: line 1: mc: command not found


## skleran-custom.yaml
apiVersion: "serving.kserve.io/v1beta1"
kind: "InferenceService"
metadata:
  name: "sklearn-custom"
  namespace: "user1"
  annotations:
    sidecar.istio.io/inject: "false"
spec:
  predictor:
    serviceAccountName: "sa"
    model:
      modelFormat:
        name: sklearn
      storageUri: "s3://kserve-test/sklearn-test/v1"

In [9]:
 from kserve import (
    KServeClient,
    utils
)
from kubernetes import client 

In [10]:
namespace = utils.get_default_target_namespace()
service_name = "sklearn-custom"
KServe = KServeClient()
KServe.get(service_name, namespace=namespace, watch=True, timeout_seconds=120)
isvc_resp=KServe.get(service_name, namespace=namespace)

NAME            READY      PREV    LATEST  URL
sklearn-custom  True          0       100  https://sklearn-custom-predictor.user1.ezua.ksi.local


In [11]:
isvc_resp

{'apiVersion': 'serving.kserve.io/v1beta1',
 'kind': 'InferenceService',
 'metadata': {'annotations': {'kubectl.kubernetes.io/last-applied-configuration': '{"apiVersion":"serving.kserve.io/v1beta1","kind":"InferenceService","metadata":{"annotations":{"sidecar.istio.io/inject":"false"},"name":"sklearn-custom","namespace":"user1"},"spec":{"predictor":{"model":{"modelFormat":{"name":"sklearn"},"storageUri":"s3://kserve-test/sklearn-test/v1"},"serviceAccountName":"sa"}}}\n',
   'sidecar.istio.io/inject': 'false'},
  'creationTimestamp': '2024-02-28T05:40:33Z',
  'finalizers': ['inferenceservice.finalizers'],
  'generation': 1,
  'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:annotations': {'.': {},
       'f:kubectl.kubernetes.io/last-applied-configuration': {},
       'f:sidecar.istio.io/inject': {}}},
     'f:spec': {'.': {},
      'f:predictor': {'.': {},
       'f:model': {'.': {},
        'f:modelFormat': 

In [12]:
X_test_list = X_test.tolist()

In [13]:
import requests
isvc_url = isvc_resp['status']['address']['url']
print(isvc_url)

http://sklearn-custom-predictor.user1.svc.cluster.local


In [14]:
sklearn_test_input={
    "instances": X_test_list
}

In [15]:
ver_info = '/v1/models/'
operaton = ':predict'
rsvc_url = isvc_url + ver_info + service_name + operaton

response = requests.post(rsvc_url, json=sklearn_test_input)

In [16]:
print(response.text)

{"predictions":[[1.825697613318415,2.6449015931558004],[1.825697613318415,2.6449015931558004],[1.825697613318415,2.6449015931558004],[1.825697613318415,2.6449015931558004],[1.825697613318415,2.6449015931558004],[1.825697613318415,2.6449015931558004],[1.825697613318415,2.6449015931558004],[1.825697613318415,2.6449015931558004],[1.825697613318415,2.6449015931558004],[1.825697613318415,2.6449015931558004],[1.825697613318415,2.6449015931558004],[1.825697613318415,2.6449015931558004],[1.825697613318415,2.6449015931558004],[1.825697613318415,2.6449015931558004],[1.825697613318415,2.6449015931558004],[1.825697613318415,2.6449015931558004],[1.825697613318415,2.6449015931558004],[1.825697613318415,2.6449015931558004],[1.825697613318415,2.6449015931558004],[1.825697613318415,2.6449015931558004],[1.825697613318415,2.6449015931558004],[1.825697613318415,2.6449015931558004],[1.825697613318415,2.6449015931558004],[1.825697613318415,2.6449015931558004],[1.825697613318415,2.6449015931558004],[1.825697

In [19]:
response.json()["predictions"] == y_3

array([[ True,  True],
       [ True,  True],
       [ True,  True],
       ...,
       [ True,  True],
       [ True,  True],
       [ True,  True]])